# Error propagation

a) Propagate uncertainties for the following expressions using [SymPy](https://www.sympy.org) following the examples for [uncorrelated variables](https://nbviewer.jupyter.org/urls/www.physi.uni-heidelberg.de/Einrichtungen/FP/Datenanalyse/FP_Gaussian_error_propagation.ipynb?flush_cache=false) and [correlated variables](https://nbviewer.jupyter.org/urls/www.physi.uni-heidelberg.de/Einrichtungen/FP/Datenanalyse/FP_Gaussian_error_propagation_corr.ipynb?flush_cache=false) from the FP web page.

i) Find expressions for the absolute uncertainty $\sigma_z$ for $z = x + y$ and $z = x - y$ 

ii) Find expressions for the relative uncertainty $\sigma_z / z$ for $z = x \cdot y, \; z = x / y$ and $z = x^n y^n$

iii) The acceleration of gravity with a simple pendulum is given by the following formula:
$$g = 4  \pi^2 \frac{L}{T^2}$$
The relevant variables are the length $L$ of the pendulum and the period $T$ with the corresponding errors $\sigma_L$ and $\sigma_T$.

b) The radius $r$ and the height $h$ of a cylinder have been measured to $r = 2$ cm and $h = 3$ cm. The uncertainty for both measurements is $\sigma = 0.05$ cm. Determine the volume of the cylinder and its uncertainty assuming (i) that both measurements are uncorrelated and (ii) that both measurements are fully correlated.

c) The scattering angle and the radial distance of a certain particle can be determined from a position measurement $(x,y)$ 
$$r = \sqrt{x^2 + y^2}, \quad \theta = \mathrm{atan2}(y, x)$$
You find more on the [atan2](https://en.wikipedia.org/wiki/Atan2) function on wikipedia. The position ($x$,$y$) is measured with the corresponding uncertainties $\sigma_x$ and $\sigma_y$. Write a python function that returns the covariance matrix $U$ of $r$ and $\theta$ for a given covariance matrix $V$ of $x$ and $y$. Determine $U$ under the assumption that $x$ and $y$ are uncorrelated. Hint: The formulas you need can be found in the script.


### Solution

In [1]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
#import matplotlib.mlab as mlab
%matplotlib inline
import numpy as np
from numpy import transpose as tp
from sympy import *
from IPython.display import display, Latex


The first step when dealing with symbolic mathematics in python is to close your python programm and instead open Mathematica.

a) First of all we will define the error propagation for uncorrelated variables. This is pretty straigthforward as we can determine the uncertainty using regular Gaussian error propagation for a functon $f(x_1, \dots, x_n)$

$$ \sigma_f = \sqrt{\sum_{i = 1}^n \left( \dfrac{\text{d} f}{\text{d} x^i} \cdot \sigma_i\right)^2 }. $$

In [2]:
'''Examples from the FP Website'''

# Error propagation from uncorrelated variables
def error_prop(f, vars, rel = False):
    """
    Symbolic Gaussian error propagation.
    
    Arguments:
    f: formula (sympy expression)
    vars: list of independent variables and corresponding uncertainties [(x1, sigma_x1), (x2, sigma_x2), ...]
    
    return value: sympy expression for the uncertainty of the calculated expression f
    
    """
    z = Symbol(str(f))
    sum = S(0) # empty SymPy expression
    for (x, sigma) in vars:
        sum += diff(f, x)**2 * sigma**2 
        
    if rel == False:
        display(Latex('${} = $'.format( '\sigma_{' + str(z) + '}') ) ) 
        return sqrt(simplify(sum))
    elif rel == True:
        display(Latex('${} = $'.format( '\dfrac{\sigma_{' + str(z) + '}}{' + str(z) + '}') ) ) 
        return sqrt(simplify(sum)) / f

# Error propagation for correlated variables
def error_prop_corr(f, x, V, rel = False):
    """
    f: function f = f(x[0], x[1], ...)
    x: list of variables
    V: covariance matrix (python 2d list)
    """
    z = Symbol(str(f))
    sum = S(0) # empty sympy expression
    for i in range(len(x)):
        for j in range(len(x)):
            sum += diff(f, x[i]) * diff(f, x[j]) * V[i][j]
    
    if rel == False:
        display(Latex('${} = $'.format( '\sigma_{' + str(z) + '}') ) ) 
        return sqrt(simplify(sum))
    elif rel == True:
        display(Latex('${} = $'.format( '\dfrac{\sigma_{' + str(z) + '}}{' + str(z) + '}') ) ) 
        return sqrt(simplify(sum)) / f
    

The way this code works is, we first have to manually define our variables and uncertainties as symbols and also define a covariance matrix. We tried coding something that would generalize this process for more dimensions and work automatically, but we didn't come to a clean solution. This code works properly for a small number of variables, but it becomes very cumbersome for larger dimensions.

i) We find expressions for the absolute uncertainty $\sigma_z$ for $z = x \pm y$.

In [3]:
# Define our variables and uncertainties
x, y, sigma_x, sigma_y, rho = symbols(['x', 'y', '\\sigma_x', '\\sigma_y', '\\rho'])

# Define covariance matrix
C = np.array([[sigma_x ** 2, rho * sigma_x * sigma_y],
              [rho * sigma_x * sigma_y, sigma_y ** 2]])

# Define function f and vars
z = x + y
vars = [x, y]

# Propagate
error_prop_corr(z, vars, C)

<IPython.core.display.Latex object>

sqrt(2*\rho*\sigma_x*\sigma_y + \sigma_x**2 + \sigma_y**2)

In [4]:
# Define new f
z = x - y
error_prop_corr(z, vars, C)

<IPython.core.display.Latex object>

sqrt(-2*\rho*\sigma_x*\sigma_y + \sigma_x**2 + \sigma_y**2)

ii) We now find expressions for the relative uncertainty $\sigma_z / z$ for $z = x \cdot y$, $z = x / y$ and $z = x^n y^n$. 

In [5]:
''' We proceed similarly to before. Since we're still dealing with two 
variables, we can keep the previous definitions'''

# Define f
z = x * y
error_prop_corr(z, vars, C, rel = True)

<IPython.core.display.Latex object>

sqrt(2*\rho*\sigma_x*\sigma_y*x*y + \sigma_x**2*y**2 + \sigma_y**2*x**2)/(x*y)

In [6]:
z = x / y
error_prop_corr(z, vars, C, rel = True)

<IPython.core.display.Latex object>

y*sqrt((-2*\rho*\sigma_x*\sigma_y*x*y + \sigma_x**2*y**2 + \sigma_y**2*x**2)/y**4)/x

In [7]:
n = Symbol('n')
z = x ** n * y ** n
error_prop_corr(z, vars, C, rel = True)

<IPython.core.display.Latex object>

sqrt(n**2*x**(2*n - 2)*y**(2*n - 2)*(2*\rho*\sigma_x*\sigma_y*x*y + \sigma_x**2*y**2 + \sigma_y**2*x**2))/(x**n*y**n)

iii) The errors for the acceleration of gravity in a simple pendulum with $g = 4 \pi^2 \dfrac{L}{T^2}$

In [8]:
# Define variables
L, T, sigma_L, sigma_T, rho = symbols(['L', 'T', '\\sigma_L', '\\sigma_T', '\\rho'])
C = np.array([[sigma_L ** 2, rho * sigma_L * sigma_T],
              [rho * sigma_L * sigma_T, sigma_T ** 2]])

g = 4 * pi ** 2 * L / (T ** 2)
vars = [L, T]

error_prop_corr(g, vars, C)

<IPython.core.display.Latex object>

4*pi**2*sqrt((4*L**2*\sigma_T**2 - 4*L*T*\rho*\sigma_L*\sigma_T + T**2*\sigma_L**2)/T**6)

In [9]:
error_prop_corr(g, vars, C, rel = True)

<IPython.core.display.Latex object>

T**2*sqrt((4*L**2*\sigma_T**2 - 4*L*T*\rho*\sigma_L*\sigma_T + T**2*\sigma_L**2)/T**6)/L

b) Now we consider a cylinder of radius $r = 2\, \mathrm{cm}$ and height $h = 3\, \mathrm{cm}$ each with an uncertainty of $\sigma = 0.05\, \mathrm{cm}$. We determine the uncertainty of the volume $V = \pi r^2 h$ assuming that i) both values are uncorrelated and ii) both values are fully correlated, i.e. $\rho = 1$. For this, we can proceed analogously to the previous task, and then input real values for our defined symbols:

i) For the case where the variables are uncorrelated:

In [10]:
# Define our variables and uncertainties
r, h, sigma, rho = symbols(['r', 'h', '\\sigma', '\\rho'])

# Define covariance matrix
C = np.array([[sigma ** 2, rho * sigma * sigma],
              [rho * sigma * sigma, sigma ** 2]])

# Define function f and vars
V = pi * r ** 2 * h
vars = [r, h]

# Propagate
error_prop_corr(V, vars, C)

<IPython.core.display.Latex object>

pi*sqrt(\sigma**2*r**2*(4*\rho*h*r + 4*h**2 + r**2))

In [11]:
sigma_V = error_prop_corr(V, vars, C).subs([(r, 2), (h, 3), 
                                            (sigma, 0.05), 
                                            (rho, 0)])
# Ignore the sigma display :)
val_V = V.subs([(r, 2), (h, 3)])

display(Latex('$V = {} \pm {}$'.format(val_V.round(1), sigma_V.round(1))))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

ii) Fully correlated

In [12]:
sigma_V = error_prop_corr(V, vars, C).subs([(r, 2), (h, 3), 
                                            (sigma, 0.05), 
                                            (rho, 1)])
# Ignore the sigma display :)
val_V = V.subs([(r, 2), (h, 3)])

display(Latex('$V = ({} \pm {})$ cm$^3$'.format(val_V.round(1), sigma_V.round(1))))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

c) Now we'll deal with the case where we have two uncorrelated variables $x, y$ with corresponding covariance matrix $\textbf{C}$ and then we perform a coordinate transformation and analyze how we can find the covariance matrix and the errors afterwards. We have

$$ r = \sqrt{x^2 + y^2} \ \ \ \ \ \ \ \ \ \ \theta = \mathrm{atan2}(x, y) = \arctan{\dfrac{y}{x}} $$.

We now want to find the covariance matrix of $r, \theta$, $\textbf{E}$. Per the script, this can be done as

$$ \textbf{E} = \textbf{G} \cdot \textbf{C} \cdot \textbf{G}^T $$

with the transformation matrix $\textbf{G}$ defined as

$$ \textbf{G}_{ij} = \dfrac{\partial f_i}{\partial x_j} $$ for the functions $f_{i} \in \{r(x, y),\, \theta(x, y)\}$.

In [13]:
# We define our variables
r, theta, x, y, sig_r, sig_theta, sig_x, sig_y = symbols(['r', '\\theta', 'x',
                                                          'y', '\\sigma_r', 
                                                         '\\sigma_\\theta',
                                                         '\\sigma_x', 
                                                          '\\sigma_y'])
rho = Symbol('\\rho')

C = np.array([[sig_x ** 2, rho * sig_x * sig_y], 
             [rho * sig_x * sig_y, sig_y ** 2]])

# And our functions
r = sqrt(x ** 2 + y ** 2)
theta = atan2(y, x)
f = Matrix([r, theta])
X = Matrix([x, y])

# We define a function to transform the covariance matrix
def cov_transform(f, vars, C):
    ''' In this format, we need to input both the function f and the 
    variables as a Matrix()
    '''
    # First determine Jacobi matrix of f
    G = f.jacobian(vars)
    # Next we transform
    return simplify(G * C * G.transpose())

cov_transform(f, X, C)

Matrix([
[       (\sigma_x*x*(\rho*\sigma_y*y + \sigma_x*x) + \sigma_y*y*(\rho*\sigma_x*x + \sigma_y*y))/(x**2 + y**2), (-\sigma_x*y*(\rho*\sigma_y*y + \sigma_x*x) + \sigma_y*x*(\rho*\sigma_x*x + \sigma_y*y))/(x**2 + y**2)**(3/2)],
[(\sigma_x*x*(\rho*\sigma_y*x - \sigma_x*y) - \sigma_y*y*(\rho*\sigma_x*y - \sigma_y*x))/(x**2 + y**2)**(3/2),     -(\sigma_x*y*(\rho*\sigma_y*x - \sigma_x*y) + \sigma_y*x*(\rho*\sigma_x*y - \sigma_y*x))/(x**2 + y**2)**2]])

For the case where $x, y$ are uncorrelated, i.e. $\rho = 0$ we can evaluate the previously determined matrix:

In [14]:
cov_transform(f, X, C).subs(rho, 0)

Matrix([
[      (\sigma_x**2*x**2 + \sigma_y**2*y**2)/(x**2 + y**2), (-\sigma_x**2*x*y + \sigma_y**2*x*y)/(x**2 + y**2)**(3/2)],
[(-\sigma_x**2*x*y + \sigma_y**2*x*y)/(x**2 + y**2)**(3/2),  -(-\sigma_x**2*y**2 - \sigma_y**2*x**2)/(x**2 + y**2)**2]])

This has non vanishing off-diagonal components due to the fact that $r, \theta$ are functions of $x, y$ and are therefore correlated.